Chargement des données csv

In [14]:
import pandas as pd #import de la bibliothèque pandas

path = "GooglePlayReviewsEN.csv" #import du fichier csv 
df_content = pd.read_csv(path) #lecture du fichier csv

Lecture du fichier csv

In [15]:
import csv #import de la bibliothèque csv

# ouverture du fichier csv en mode lecture 
with open(path, newline='',  encoding='utf-8') as csvfile:
    # création d'un objet reader qui va lire le fichier csv
    reader = csv.reader(csvfile)
    # création d'une liste qui va contenir les lignes du fichier csv
    string_list = []
    # boucle sur les lignes du fichier csv
    for row in reader:
        # ajout de la ligne dans la liste
        string_list.append(row[0])

Segmentation des données de la liste

In [16]:
string_list = string_list[:1000] #on ne prend que les 1000 premiers commentaires

Traitement des données sur le fichier jsonlines

!pip install jsonlines

In [ ]:
#import jsonl & fichier annotations
import jsonlines

path = "AnnotedComments.jsonl"
annotations =[]


with jsonlines.open(path) as reader:
  i = 0
  for obj in reader:
    annotation = [i]
    for note in obj['entities']:
      annotation.append([note['start_offset'], note['end_offset']])
      print(annotation)
    if obj['entities']:
      annotations.append(annotation)
    i += 1



Pre-traitement du texte

In [ ]:
# import de la bibliothèque spacy 
import spacy
# import de la bibliothèque re 
import re

# chargement du modèle en_core_web_sm 
!python -m spacy download en_core_web_sm

# chargement du modèle anglais de spacy 
nlp = spacy.load("en_core_web_sm")

# liste des mots d'arrêt les plus courants en anglais
listStopWords = [str(x) for x in nlp.Defaults.stop_words]

Création de la fonction de pré-traitement des textes

In [19]:
def united(text):

    """Suppression des explications"""
    doc =  text.split(" ");
    # Itérer à travers chaque token dans le document
    for token in doc:
        # Si le token est "parce que", "car" ou "puisque", supprimer tous les tokens suivants dans la phrase
        if token in ["because", "since", "as", ]:
            # Supprimer tous les tokens après le token "parce que", "car" ou "puisque"
            doc = doc[:doc.index(token)]
            # Sortir de la boucle
            break
    
    # Rejoindre les tokens dans la phrase
    sentence = ' '.join(doc)
    """Fin de la suppression des explications"""

    """Suppression des caractères spéciaux"""
    sentence = re.sub(r'\([^()]*\)', '', sentence) # Suppression des parenthèses

    """Suppression des liens web dans la phrase"""
    sentence = re.sub('http[s]?://\S+', '', sentence) # Suppression des liens web


    """Suppression des numéros de téléphone et des adresses e-mail"""
    quotes = r'\"[^\"]+\"' # expression régulière pour les citations
    phone_regex = r"(?<!\d)(?:\d[ -/\\_\d]*){10}(?!\d)" # Expression régulière pour les numéros de téléphone
    email_regex = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"  # Expression régulière pour les adresses e-mail

    sentence = re.sub(quotes,"", sentence) # Suppression des citations
    sentence = re.sub(phone_regex,"", sentence) # Suppression des numéros de téléphone
    sentence = re.sub(email_regex,"", sentence) # Suppression des adresses e-mail
    sentence = re.sub(r'[^\w]', ' ', sentence) # Suppression des caractères spéciaux

    """Suppression des mots d'arrêt"""
    doc = sentence.split(" ") # Séparer la phrase en tokens
    for word in doc: # Itérer à travers chaque token dans la phrase
        if word in listStopWords: # Si le token est un mot d'arrêt, supprimer le token
            doc.remove(word) # Supprimer le token
    
    sentence = ' '.join(doc) # Rejoindre les tokens dans la phrase


    """Suppression des caractéres spéciaux "'" """
    sentence = sentence.replace("'", " ") # Remplacer les caractéres spéciaux "'" par un espace

    """Suppression des mots de moins de 2 caractères"""
    words = sentence.split(" ") # Séparer la phrase en tokens
    newSentence = "" # Créer une nouvelle phrase vide	
    for word in words: # Itérer à travers chaque token dans la phrase
        if len(word) > 1: # Si le token est plus grand que 1 caractère, ajouter le token à la nouvelle phrase
            newSentence += word + " " # Ajouter le token à la nouvelle phrase
    sentence = newSentence # Mettre la nouvelle phrase dans la variable sentence
    
    """Suppression des pronoms et déterminants"""
    doc = nlp(sentence) # Charger la phrase dans le modèle spacy
    text = "" # Créer une nouvelle phrase vide
    for token in doc: # Itérer à travers chaque token dans le document
        if (token.pos_ == "PRON" or token.pos_ == "DET"): # Si le token est un pronom ou un déterminant, ignorer le token
            continue # Ignorer le token
        else: # Sinon, ajouter le token à la nouvelle phrase
            if not token.text.__contains__("'"): # Si le token ne contient pas le caractére spécial "'", ajouter le token à la nouvelle phrase
                text += token.text + " " # Ajouter le token à la nouvelle phrase
                text += " " # Ajouter un espace à la nouvelle phrase
    sentence = text # Mettre la nouvelle phrase dans la variable sentence

    """Suppression des noms propres et des noms d'organisations"""
    doc = nlp(sentence) # Charger la phrase dans le modèle spacy
    result = [] # Créer une nouvelle phrase vide 
    for token in doc: # Itérer à travers chaque token dans le document
        if not (token.ent_type_ == "ORG" or token.ent_type_ == "PERSON" or token.text.istitle()): # Si le token est un nom propre ou un nom d'organisation, ignorer le token
            result.append(token.text) # Ajouter le token à la nouvelle phrase
    sentence =  " ".join(result) # Rejoindre les tokens dans la phrase

    return sentence # Retourner la phrase nettoyée

Application de la fonction de pré-traitement sur les données

In [20]:
united_string = [] # Créer une liste vide
for i in range(len(string_list)): # Itérer à travers chaque phrase dans la liste
  united_string += [united(string_list[i])] # Ajouter la phrase nettoyée à la liste

Application des patterns de la méthode SAFE

In [21]:
def superpose(da, fa, db, fb):
  return (da <= db <= fa | da <= fb <= fa | (db < da & fa < fb))


def matchTokens(id, start, end):
  both = 0
  for labeled in annotations:
    if id == labeled[0]:
      for i in range(1, len(labeled)):
        print(labeled[i][0], labeled[i][1], start, end)
        if superpose(labeled[i][0], labeled[i][1], start, end):
          both +=1
          break
  return both



Création des matchers de spaCy

In [23]:
from spacy.matcher import Matcher # import matcher 

matcher1 = Matcher(nlp.vocab) # création d'un matcher 
matcher2 = Matcher(nlp.vocab) # création d'un matcher
matcher3 = Matcher(nlp.vocab) # création d'un matcher


# Add match ID "HelloWorld" with no callback and one pattern
pattern1 = [{"POS": "NOUN"}, {"POS": "NOUN"}]
pattern2 = [{"POS": "VERB"}, {"POS": "NOUN"}]
pattern3 = [{"POS": "ADJ"}, {"POS": "NOUN"}]
pattern4 = [{"POS": "NOUN"}, {"POS": "CCONJ"}, {"POS": "NOUN"}]
pattern5 = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern6 = [{"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern7 = [{"POS": "VERB"}, {"POS": "PRON"}, {"POS": "NOUN"}]
pattern8 = [{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern9 = [{"POS": "VERB"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern10 = [{"POS": "ADJ"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern11 = [{"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
pattern12 = [{"POS": "VERB"}, {"POS": "DET"}, {"POS": "NOUN"}]
pattern13 = [{"POS": "VERB"}, {"POS": "NOUN"}, {"POS": "ADP"}, {"POS": "NOUN"}]
pattern14 = [{"POS": "ADJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
pattern15 = [{"POS": "ADJ"}, {"POS": "CONJ"}, {"POS": "ADJ"}]
pattern16 = [{"POS": "VERB"}, {"POS": "ADP"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern17 = [{"POS": "VERB"}, {"POS": "PRON"}, {"POS": "ADJ"}, {"POS": "NOUN"}]
pattern18 = [{"POS": "NOUN"}, {"POS": "CCONJ"}, {"POS": "NOUN"}, {"POS": "NOUN"}]
matcher1.add("18", [pattern18])
matcher1.add("17", [pattern17])
matcher1.add("16", [pattern16])
matcher2.add("15", [pattern15])
matcher1.add("14", [pattern14])
matcher1.add("13", [pattern13])
matcher2.add("12", [pattern12])
matcher2.add("11", [pattern11])
matcher2.add("10", [pattern10])
matcher2.add("9", [pattern9])
matcher2.add("8", [pattern8])
matcher2.add("7", [pattern7])
matcher2.add("6", [pattern6])
matcher2.add("5", [pattern5])
matcher2.add("4", [pattern4])
matcher3.add("3", [pattern3])
matcher3.add("2", [pattern2])
matcher3.add("1", [pattern1])

matched_id_list = []
matched_list = []
#doc = nlp("Song and artist album,  Create your greatest album, ")

for row in range(len(united_string)):
  doc = nlp(united_string[row])
  while(True):
    matches = matcher1(doc)
    for match_id, start, end in matches:
      trupos = matchTokens(row, start, end)
      string_id = nlp.vocab.strings[match_id]
      nlp_list = list(doc)
      #print(doc[start:end])
      matched_list.append(doc[start:end])
      matched_id_list.append(string_id)

      for i in range(end-start):
        if(start<len(nlp_list)):
          del nlp_list[start]

      doc = nlp(" ".join([e.text for e in nlp_list]))
    if (not matches):
      break
  while(True):
    matches = matcher1(doc)
    #trupos += matchTokens(row, matches)
    for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]
      nlp_list = list(doc)
      print(doc[start:end])
      matched_list.append(doc[start:end])
      matched_id_list.append(string_id)

      for i in range(end-start):
        if(start<len(nlp_list)):
          del nlp_list[start]

      doc = nlp(" ".join([e.text for e in nlp_list]))
    if (not matches):
      break
  while(True):
    matches = matcher1(doc)
    #trupos += matchTokens(row, matches)
    for match_id, start, end in matches:
      string_id = nlp.vocab.strings[match_id]
      nlp_list = list(doc)
      print(doc[start:end])
      matched_list.append(doc[start:end])
      matched_id_list.append(string_id)

      for i in range(end-start):
        if(start<len(nlp_list)):
          del nlp_list[start]

      doc = nlp(" ".join([e.text for e in nlp_list]))
    if (not matches):
      break

#print(trupos)
#Remove duplicate and synonyms, noise
print(matched_list)

[]


In [24]:
for el in matched_list:
  if len(el.text.split(" "))<=1:
    matched_list.remove(el)

Affichage graphique de thème des caractéristiques

In [ ]:
matched_list_str = [str(k) for k in matched_list] # Conversion des phrases en string

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer # import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation # import LatentDirichletAllocation
import matplotlib.pyplot as plt # import matplotlib
import networkx as nx # import networkx

# Liste des caractéristiques à traiter
features = matched_list_str[:100]  # 100 caractéristiques à traiter

# Preprocessing des données
preprocessed_features = [feat.lower().split() for feat in features]

# Création de la matrice de termes
vectorizer = CountVectorizer()
dtm = vectorizer.fit_transform([' '.join(feat) for feat in preprocessed_features]) # Création de la matrice de termes

# Identification des thèmes de la matrice de termes
n_topics = 10 # Nombre de thèmes
lda = LatentDirichletAllocation(n_components=n_topics, random_state=0) # Création du modèle LDA
doc_topic_dist = lda.fit_transform(dtm) # Identification des thèmes de la matrice de termes

# Création du graphe des thèmes
G = nx.Graph() # Création du graphe
for i, feat in enumerate(features): # Pour chaque caractéristique
    topic = doc_topic_dist[i].argmax() # Récupération du thème de la caractéristique
    G.add_node(feat, topic=topic, prob=doc_topic_dist[i][topic]) # Ajout du noeud correspondant à la caractéristique

# Ajout des arêtes entre les noeuds
for u, v in G.edges():  # Pour chaque paire de noeuds
    if G.nodes[u]['topic'] == G.nodes[v]['topic']: # Si les noeuds sont du même thème
        G.add_edge(u, v) # Ajout de l'arête entre les noeuds

# Affichage des noeuds les plus importants par thème
top_n = 100
for topic in range(n_topics): # Pour chaque thème
    nodes = [node for node in G.nodes() if G.nodes[node]['topic'] == topic] # Récupération des noeuds du thème
    nodes_sorted = sorted(nodes, key=lambda x: G.nodes[x]['prob'], reverse=True) # Tri des noeuds par probabilité
    nodes_top = nodes_sorted[:top_n] # Récupération des noeuds les plus importants
    for node in nodes_top: # Pour chaque noeud
        G.nodes[node]['label'] = node.split()[-1] # Ajout du label correspondant au dernier mot du noeud

# Affichage du graphe des thèmes
pos = nx.spring_layout(G, k=0.5, iterations=50) # Positionnement des noeuds
colors = plt.cm.get_cmap('hsv', n_topics) # Couleurs des noeuds
for topic in range(n_topics): # Pour chaque thème
    nodes = [node for node in G.nodes() if G.nodes[node]['topic'] == topic] # Récupération des noeuds du thème
    nodes_top = [node for node in nodes if 'label' in G.nodes[node]] # Récupération des noeuds les plus importants
    node_colors = [colors(topic) for _ in range(len(nodes_top))] # Couleurs des noeuds
    nx.draw_networkx_nodes(G, pos, nodelist=nodes_top, node_color=node_colors, alpha=0.8) # Affichage des noeuds
nx.draw_networkx_edges(G, pos, edge_color='gray', alpha=0.5) # Affichage des arêtes 
node_labels = nx.get_node_attributes(G, 'label') # Récupération des labels des noeuds
nx.draw_networkx_labels(G, pos, labels=node_labels, font_size=10, font_family='sans-serif') # Affichage des labels
plt.axis('off') # Suppression des axes
plt.figure(figsize=(30, 20)) # Taille de la figure 
plt.show() # Affichage du graphe


III. L'appariement des textes

In [ ]:
!pip install scikit-learn # installation de scikit-learn
!pip install -U sentence-transformers # installation de sentence-transformers 

In [ ]:
from sentence_transformers import SentenceTransformer # import de sentence-transformers
model = SentenceTransformer('all-MiniLM-L6-v2') # chargement du modèle

In [ ]:
!pip install faiss-cpu # installation de faiss-cpu
import faiss # import de faiss

In [ ]:
import numpy as np

In [ ]:
from sklearn.metrics import silhouette_score

Modèle des K-Means

In [ ]:
from sklearn.cluster import KMeans # import de KMeans

# Définition du nombre de clusters à créer 
num_clusters = 63

# Encodage des phrases en vecteurs 
phrase_embeddings = model.encode(matched_list)

# Création du modèle KMeans 
kmeans = KMeans(n_clusters=num_clusters, random_state=0).fit(phrase_embeddings)

# Calcul de la silhouette score 
silhouette_avg = silhouette_score(phrase_embeddings, kmeans.labels_)

print("Le score silhouette vaut :", silhouette_avg)

# Création des clusters 
clusters = [[] for i in range(num_clusters)] # Clusters
for i, label in enumerate(kmeans.labels_): # Pour chaque phrase et son cluster associé 
    clusters[label].append(matched_list[i]) # Ajout de la phrase au cluster correspondant

# Affichage des clusters
for i in range(num_clusters):
    print("Cluster {}:".format(i+1))
    print(clusters[i])


Modèle Agglomerative Clustering

In [ ]:
# Import de AgglomerativeClustering
from sklearn.cluster import AgglomerativeClustering

# Encodahe des phrases en vecteurs
corpus_embeddings = model.encode(matched_list)

# Normalisation des vecteurs
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Création du modèle AgglomerativeClustering
clustering_model = AgglomerativeClustering(n_clusters=None, distance_threshold=1.5) 
# distance_threshold : distance maximale entre deux clusters pour qu'ils soient fusionnés
clustering_model.fit(corpus_embeddings) # Evaluation du modele 
cluster_assignment = clustering_model.labels_ # Assignation des phrases aux clusters

# Calcul de la silhouette score
silhouette_avg = silhouette_score(corpus_embeddings, cluster_assignment)

print("Le score silhouette vaut :", silhouette_avg) # Affichage du score silhouette

# Création des clusters
clustered_sentences = {} # Dictionnaire des clusters
for sentence_id, cluster_id in enumerate(cluster_assignment): # Pour chaque phrase et son cluster associé
    if cluster_id not in clustered_sentences: # Si le cluster n'existe pas encore
        clustered_sentences[cluster_id] = [] # Création du cluster
    clustered_sentences[cluster_id].append(matched_list[sentence_id]) # Ajout de la phrase au cluster correspondant

for i, cluster in clustered_sentences.items(): # Pour chaque cluster et ses phrases associées
    print("Cluster ", i+1) # Affichage du numéro du cluster
    print(cluster) # Affichage des phrases du cluster
    print("") # Saut de ligne

Modèle de DBScan sans FAISS

In [ ]:
from sklearn.cluster import DBSCAN # import de DBSCAN

# Encodage des phrases en vecteurs
corpus_embeddings = model.encode(matched_list)

# Normalisation des vecteurs
corpus_embeddings = corpus_embeddings /  np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Création du modele DBSCAN
clustering_model = DBSCAN(eps=0.5, min_samples=2, metric='euclidean')

# Assignation des phrases aux clusters
cluster_assignment = clustering_model.fit_predict(corpus_embeddings)

# Calcul de la silhouette score
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

# Affichage du score silhouette
print("Le score silhouette vaut :", silhouette_avg)

clustered_sentences = {} # Création des clusters
for sentence_id, cluster_id in enumerate(cluster_assignment): # Pour chaque phrase et son cluster associé 
    if cluster_id not in clustered_sentences: # Si le cluster n'existe pas encore
        clustered_sentences[cluster_id] = [] # Création du cluster
    clustered_sentences[cluster_id].append(matched_list[sentence_id]) # Ajout de la phrase au cluster correspondant

# Affichage des clusters
for i, cluster in clustered_sentences.items():
    print("Cluster ", i+1) # Affichage du numéro du cluster
    print(cluster) # Affichage des phrases du cluster
    print("") # Saut de ligne


Modèle DBScan avec FAISS

In [ ]:
from sklearn.cluster import DBSCAN # import de DBSCAN 

# Encodage des phrases en vecteurs 
corpus_embeddings = model.encode(matched_list)

# Normalisation des vecteurs
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Création des index FAISS pour la recherche des plus proches voisins
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings)

# Recherche des plus proches voisins 
_, indices = index.search(corpus_embeddings, k=5)  # 5 plus proches voisins

# Création du modèle DBSCAN 
clustering_model = DBSCAN(eps=0.42, min_samples=5, metric='euclidean')

#clustering_model = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_epsilon=0.40)

# Assignation des phrases aux clusters 
cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])

# Calcul de la silhouette score
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)

print("Le score silhouette vaut :", silhouette_avg)

# Création des clusters
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment):
    if cluster_id not in clustered_sentences: # Si le cluster n'existe pas encore
        clustered_sentences[cluster_id] = [] # Création du cluster
    clustered_sentences[cluster_id].append(matched_list[sentence_id]) # Ajout de la phrase au cluster correspondant

# Affichage des clusters
for i, cluster in clustered_sentences.items(): # Pour chaque cluster et ses phrases associées
    print("Cluster ", i+1) # Affichage du numéro du cluster
    print(cluster) # Affichage des phrases du cluster 
    print("") # Saut de ligne


Affichage graphique du modèle de DBScan

In [ ]:
import matplotlib.pyplot as plt # import matplotlib
from sklearn.decomposition import PCA # import de PCA

# Encodage des phrases en vecteurs
corpus_embeddings = model.encode(matched_list)

# Normalisation des vecteurs 
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Réglage de FAISS pour la recherche des plus proches voisins
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])

# Ajout des vecteurs au modèle FAISS 
index.add(corpus_embeddings)

# Recherche des 5 plus proches voisins pour chaque phrase
_, indices = index.search(corpus_embeddings, k=5)  # 5 plus proches voisins 

# Création du modèle DBSCAN
clustering_model = DBSCAN(eps=1.5, min_samples=2, metric='euclidean')

# Assignation des phrases aux clusters
cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]])

# Réduction de la dimension des vecteurs à 2
pca = PCA(n_components=2)
corpus_embeddings_2d = pca.fit_transform(corpus_embeddings) # Réduction de la dimension des vecteurs à 2

# Affichage des clusters
fig, ax = plt.subplots() # Création de la figure
scatter = ax.scatter(corpus_embeddings_2d[:, 0], corpus_embeddings_2d[:, 1], c=cluster_assignment) # Affichage des points
legend1 = ax.legend(*scatter.legend_elements(), 
                    loc="upper right", title="Clusters") # Affichage de la légende
ax.add_artist(legend1) # Ajout de la légende à la figure
plt.show() # Affichage de la figure


Modèle HDBScan

In [ ]:
!pip install hdbscan # installation de hdbscan

In [ ]:
# Import des librairies nécessaires
import hdbscan

# Encodage des données en vecteurs
corpus_embeddings = model.encode(matched_list)

# Normalisation des vecteurs
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Réglage de FAISS pour la recherche des plus proches voisins
index = faiss.IndexFlatIP(corpus_embeddings.shape[1])
index.add(corpus_embeddings) # Ajout des vecteurs au modèle FAISS

# Recherche des 5 plus proches voisins pour chaque phrase
_, indices = index.search(corpus_embeddings, k=5)  # 5 plus proches voisins

# Création du modèle HDBSCAN
clustering_model = hdbscan.HDBSCAN(min_cluster_size=5, metric='euclidean', cluster_selection_epsilon=0.40)
cluster_assignment = clustering_model.fit_predict(corpus_embeddings[indices[:, 1]]) # Assignation des phrases aux clusters

# Calcul de la silhouette score
silhouette_avg = silhouette_score(corpus_embeddings[indices[:, 1]], cluster_assignment)
# Affichage du score silhouette
print("Le score silhouette vaut :", silhouette_avg)

# Affichage des clusters
clustered_sentences = {}
for sentence_id, cluster_id in enumerate(cluster_assignment): # Pour chaque phrase et son cluster associé
    if cluster_id not in clustered_sentences: # Si le cluster n'existe pas encore
        clustered_sentences[cluster_id] = [] # Création du cluster
    clustered_sentences[cluster_id].append(matched_list[sentence_id]) # Ajout de la phrase au cluster correspondant

for i, cluster in clustered_sentences.items(): # Affichage des clusters
    print("Cluster ", i+1) # Affichage du numéro du cluster
    print(cluster) # Affichage des phrases du cluster
    print("") # Saut de ligne
